### Pre -work : 


In [ ]:
rsync -avzhe ssh \
/media/concha-eloko/Linux/PPT_clean/in_vitro/in_vitro_DFs \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023

***

In [3]:
# Torch geometric modules
from torch_geometric.data import HeteroData, DataLoader
import torch_geometric.transforms as T
from torch_geometric.nn import to_hetero , HeteroConv , GATv2Conv
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import LinkNeighborLoader

# Torch modules
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

# SKlearn modules
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder , label_binarize , OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score , matthews_corrcoef

# Ground modules
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import product
import random
from collections import Counter
import warnings
import logging
from multiprocessing.pool import ThreadPool

# TropiGAT modules
import TropiGAT_graph
import TropiGAT_models

warnings.filterwarnings("ignore")


In [4]:
# *****************************************************************************
# Load the Dataframes :
path_work = "/media/concha-eloko/Linux/PPT_clean"
DF_info = pd.read_csv(f"{path_work}/TropiGATv2.final_df_v2.filtered.tsv", sep = "\t" ,  header = 0)
DF_info_lvl_0 = DF_info.copy()

# Log file : 
#path_ensemble = f"{path_work}/train_nn/ensemble_2812"
#path_finetuned = f"{path_work}/train_nn/fine_tuning/models_0801"
#path_log = f"{path_work}/train_nn/fine_tuning/models_0801"


df_prophages = DF_info_lvl_0.drop_duplicates(subset = ["Phage"])
dico_prophage_count = dict(Counter(df_prophages["KL_type_LCA"]))

KLtypes = [kltype for kltype in dico_prophage_count if dico_prophage_count[kltype] >= 20]


> Open the DF

Fix Ferriol names : 

In [5]:
path_project = "/media/concha-eloko/Linux/77_strains_phage_project"
path_Dpo_domain_org = "/media/concha-eloko/Linux/depolymerase_building/clean_77_phages_depo"
path_vitro = "/media/concha-eloko/Linux/PPT_clean/in_vitro"

# ESM2 embedding dataframes :
# Ferriol 
dpo_embeddings = pd.read_csv(f"{path_project}/rbp_work/Dpo_domains_77.esm2.embedding.1512.csv", sep = "," , header = None)
dpo_embeddings = dpo_embeddings.drop([1281] , axis = 1)
dpo_embeddings.set_index([0], inplace = True)
dpo_embeddings.index = [x if x.count("__cds") > 0 else x.replace("_cds", "__cds") for x in dpo_embeddings.index]
dpo_embeddings.index = [x.replace("__CDS","__cds") if x.count("__CDS") > 0 else x for x in dpo_embeddings.index]
#dpo_embeddings.index = ['__'.join(x".split("__")[0:-1])+'_Dpo_domain' if x.count("_Dpo_domain") == 0 else x for x in dpo_embeddings.index]
dpo_embeddings.index = ['__'.join(x.split("__")[0:-1]) if x.count("_Dpo_domain") == 0 else x for x in dpo_embeddings.index]

# Beamud  
bea_embeddings = pd.read_csv(f"{path_vitro}/Bea_phages.esm2.embedding.csv", sep = "," , header = None)
bea_embeddings = bea_embeddings.drop([1281] , axis = 1)
bea_embeddings[0] = bea_embeddings[0].apply(lambda x : x.replace("_", "__"))
bea_embeddings.set_index([0], inplace = True)

# Towndsend  :
towndsend_embeddings = pd.read_csv(f"{path_vitro}/Townsed_phages.esm2.embedding.csv", sep = "," , header = None)
towndsend_embeddings = towndsend_embeddings.drop([1281] , axis = 1)
towndsend_embeddings[0] = towndsend_embeddings[0].apply(lambda x : x.replace("_", "__"))
towndsend_embeddings.set_index([0], inplace = True)

# ==> DF embeddings 
df_embeddings = pd.concat([towndsend_embeddings, bea_embeddings, dpo_embeddings], axis = 0)
# ************************************
# The matrices : 
# Beamud matrix :
bea_df = pd.read_csv(f"{path_vitro}/fine_tuning/bea_fine_tuning.df", sep = "\t", header = 0)
bea_df["Protein"] = bea_df["Protein"].apply(lambda x : x.replace("_", "__"))
pool_bea = set([kltype.strip() for kltypes in bea_df["Target"] for kltype in kltypes.split(",") if kltype.count("wzi") == 0 if kltype.count("pass") == 0])

# Ferriol matrix
ferriol_df = pd.read_csv(f"{path_vitro}/fine_tuning/ferriol_fine_tuning.df", sep = "\t", header = 0)
ferriol_df["Target"] = ferriol_df["Target"].apply(lambda x : x.replace("K", "KL"))
pool_ferriol = set([kltype.strip() for kltypes in ferriol_df["Target"] for kltype in kltypes.split(",") if kltype.count("wzi") == 0 if kltype.count("pass") == 0])

# Towndsend matrix :
towndsend_df = pd.read_csv(f"{path_vitro}/fine_tuning/towndsend_fine_tuning.df", sep = "\t", header = 0)
towndsend_df["Protein"] = towndsend_df["Protein"].apply(lambda x : x.replace("_", "__"))
pool_towndsend = set([kltype.strip() for kltypes in towndsend_df["Target"] for kltype in kltypes.split(",") if kltype.count("wzi") == 0 if kltype.count("pass") == 0])

# ==> dico Data
dico_matrices = {"ferriol" : {"matrix" : ferriol_df, "pool" : pool_ferriol}, 
                 "bea" : {"matrix": bea_df, "pool" : pool_bea}, 
                 "towndsend" : {"matrix" : towndsend_df, "pool" : pool_towndsend}}

pools_kltypes = set()
pools_kltypes.update(pool_ferriol)
pools_kltypes.update(pool_bea)
pools_kltypes.update(pool_towndsend)
pools_kltypes = list(pools_kltypes)


In [6]:
none_beta_helix = ['A1a_00002', 'A1b_00036', 'A1c_00046', 'A1d_00013', 'A1e_00024', 'A1f_00024', 'A1g_00045', 'A1h_00009', 'A1i_00037', 'A1j_00002', 'A1k_00018', 'A1l_00005', 'A1m_00045', 'A1n_00050', 'A1o_00041', 'A1p_00055', 'A1q_00023', 'A1r_00013', 'A2a_00010', 'A2a_b_00036', 'A2b_00022', 'A3a_00045', 'A3b_00016', 'A3c_00039', 'A3d_00036', 'BLCJPOBP_00041', 'DIMCIIMF_00039', 'DIMCIIMF_00240', 'DJLANJJD_00238', 'EHPPICDA_00095', 'EKPIEFBL_00113', 'EKPIEFBL_00177', 'EONHMLJF_00087', 'FADJDIKG_00083', 'FADJDIKG_00019', 'GCLPFEGH_00240', 'HIIECEMK_00043', 'INMBNBLA_00202', 'INMBNBLA_00138', 'IPGJFKPH_00198', 'K10PH82C1__cds_45', 'K11PH164C1__cds_39', 'K12P1_1__cds_43', 'K13PH07C1L__cds_54', 'K13PH07C1S__cds_53', 'K15PH90__cds_49', 'K16PH164C3__cds_43', 'K19PH14C4P1__cds_43', 'K19PH14C4P1__cds_48', 'K1PH164C1__cds_53', 'K2064PH2__cds_25', 'K21lambda1__cds_28', 'K22PH164C1__cds_50', 'K24PH164C1__cds_55', 'K25PH129C1__cds_56', 'K26PH128C1__cds_44', 'K27PH129C1__cds_43', 'K28PH129__cds_24', 'K29PH164C1__cds_25', 'K31PH164__cds_25', 'K33PH14C2__cds_25', 'K35PH164C3__cds_43', 'K37PH164C1__cds_41', 'K39PH122C2__cds_50', 'K40PH129C1__cds_52', 'K41P2__cds_11', 'K42PH8__cds_43', 'K43PH164C1__cds_35', 'K44PH129C1__cds_45', 'K48PH164C1__cds_43', 'K4PH164__cds_22', 'K50PH164C1__cds_28', 'K51PH129C1__cds_56', 'K52PH129C1__cds_25', 'K56PH164C1__cds_43', 'K58PH129C2__cds_40', 'K59PH2__cds_46', 'K61PH164C1__cds_52', 'K65PH164__cds_12', 'K66PH128C1__cds_55', 'K69PH164C2__cds_24', 'K71PH129C1__cds_51', 'K72PH164C2__cds_55', 'K72PH164C2__cds_51', 'K74PH129C2__cds_46', 'K82P1__cds_40', 'K8PH128__cds_41', 'K9PH25C2__cds_76', 'LLOFRUDD_00039', 'M5a_00175', 'NEBGPKAL_00013', 'OBHDAGOG_00029', 'POFPNCPI_00246']

df_embeddings = df_embeddings[~df_embeddings.index.isin(none_beta_helix)]


In [7]:
df_embeddings

,1,2,3,4,5,6,7,8,9,10,...,1271,1272,1273,1274,1275,1276,1277,1278,1279,1280
PP187__gp233,0.016811,0.073770,-0.009818,-0.036033,-0.085370,-0.016310,0.007661,-0.083562,0.072318,0.050186,...,-0.004649,-0.006715,0.121524,-0.004321,0.019053,0.074084,0.042733,-0.064831,0.040088,0.062354
LCALLHIG__00050,0.015476,0.014564,0.016806,-0.042899,-0.071834,0.021123,0.059287,-0.088285,0.081127,0.022274,...,0.016373,-0.006337,0.073145,-0.024415,0.028782,0.048048,0.012546,-0.085356,-0.006697,0.106924
NBNDMPCG__00165,0.097851,-0.005817,0.044202,0.066516,-0.062639,0.012461,-0.056078,-0.046124,0.025547,0.090553,...,0.091665,0.004563,0.011115,0.032996,0.009146,0.084371,0.129982,-0.089394,-0.080854,-0.026717
EKPIEFBL__00177,0.062666,-0.010341,-0.000893,0.034539,-0.107698,0.021799,-0.030059,-0.039033,-0.016901,0.012905,...,0.085217,-0.028300,0.048211,0.066592,0.019395,0.056614,0.101654,-0.031980,-0.056580,0.012406
NBNDMPCG__00161,0.003200,0.046110,0.004314,-0.063110,-0.094585,-0.018906,-0.011139,-0.041858,0.066822,0.052831,...,-0.010727,-0.004597,0.068099,-0.013416,0.020944,0.050755,0.065946,-0.076776,0.066887,0.054853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
K17alfa62__cds_66,-0.029530,0.059893,0.012090,-0.023812,-0.100989,0.049208,0.070693,-0.122688,0.029461,0.037515,...,-0.020068,-0.032618,0.111752,-0.004208,0.009533,0.094001,0.028300,-0.104947,0.032551,0.117055
K80PH1317a__cds_47,0.013453,-0.039767,-0.055876,0.069519,-0.147644,0.067943,0.098398,-0.174066,0.004090,-0.001659,...,0.018430,-0.033284,0.020107,0.003742,-0.062657,-0.019130,0.037125,-0.078332,-0.130844,0.145029
K42PH8__cds_48,-0.002214,0.075294,-0.005191,-0.019837,-0.119218,-0.002820,0.036694,-0.079576,0.067217,0.011143,...,-0.000534,-0.002793,0.089157,0.012454,0.026019,0.078977,0.057343,-0.071425,0.058036,0.034123
K2069PH1__cds_25,0.024761,0.068158,0.031650,0.050696,-0.098533,-0.003535,0.039712,-0.078981,0.056933,0.061923,...,-0.033775,-0.070897,0.087817,0.084919,0.101386,-0.049455,0.057586,-0.098228,0.045679,0.139675


In [15]:
#dico_matrices["ferriol"]
dpo_embeddings

,1,2,3,4,5,6,7,8,9,10,...,1271,1272,1273,1274,1275,1276,1277,1278,1279,1280
K15PH90__cds_55_Dpo_domain,-0.028760,0.046677,-0.010773,0.028452,-0.090442,0.027041,0.004249,-0.083708,0.022172,0.090119,...,-0.032166,-0.012386,0.079159,0.012298,0.027317,0.037254,0.069599,-0.097522,0.067495,0.062502
K80PH1317b__cds_54_Dpo_domain,0.007689,0.036850,-0.006928,-0.056424,-0.090723,0.018707,0.014913,-0.070090,0.073792,0.055322,...,0.017004,-0.000657,0.059184,-0.006782,0.023955,0.035585,0.048035,-0.081247,0.043776,0.118674
K64PH164C4__cds_24_Dpo_domain,0.015762,0.062429,-0.003427,-0.003609,-0.101109,0.028121,0.004342,-0.096114,0.062562,0.027864,...,0.042829,0.029579,0.095159,0.024894,0.002837,0.046701,0.062497,-0.084956,0.027426,0.051051
K5lambda5__cds_196_Dpo_domain,0.040111,0.046436,-0.012045,-0.043877,-0.100054,-0.028328,0.028640,-0.047144,0.065727,0.047312,...,0.017347,0.008084,0.096149,-0.031008,0.040423,0.082593,0.050161,-0.105612,0.023642,0.081104
K11PH164C1__cds_46_Dpo_domain,0.017319,0.077582,-0.001212,-0.030026,-0.070916,-0.011639,0.006673,-0.078486,0.072836,0.046921,...,0.005777,-0.032808,0.099658,-0.028466,0.020794,0.082009,0.030658,-0.091195,0.047744,0.105303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
K24PH164C1__cds_55,0.039957,-0.007577,-0.037035,0.054355,-0.128855,0.044301,0.099503,-0.163834,0.076119,-0.003351,...,0.016038,0.009034,0.027130,0.023492,-0.033999,-0.025833,0.030128,-0.042748,-0.122199,0.124239
K42PH8__cds_43,0.015222,-0.034378,-0.049853,0.063044,-0.158600,0.051105,0.079608,-0.183104,0.016171,-0.001478,...,0.023938,-0.037645,0.006023,0.016808,-0.034763,-0.044617,0.048723,-0.071603,-0.118990,0.130576
K10PH82C1__cds_45,0.017037,-0.028448,-0.066188,0.038587,-0.124722,0.051205,0.086921,-0.141901,0.024835,-0.017928,...,0.022757,-0.000056,0.013690,0.011025,-0.067530,-0.004214,0.050276,-0.072483,-0.137275,0.142341
K44PH129C1__cds_45,0.010439,-0.023608,-0.016569,0.054134,-0.148860,0.043717,0.065837,-0.149970,0.020374,-0.011850,...,0.042593,-0.044361,-0.003943,0.033981,-0.043663,-0.035262,0.058799,-0.032766,-0.110497,0.138307


> Functions :

In [11]:
# Functions : 
def finetune_kltype_df(kltype) :
    positive_lines , negative_lines = [], []
    for author in dico_matrices :
        if kltype in dico_matrices[author]["pool"] : 
            for _,row in dico_matrices[author]["matrix"].iterrows() :
                if row["Target"].count("pass") == 0 :
                    targets = [kltype.strip() for kltype in row["Target"].split(",") if kltype.count("wzi") == 0 if kltype.count("pass") == 0]
                    if kltype not in targets :
                        negative_lines.append(list(row.values))
                    else :
                        positive_lines.append(list(row.values))
    lines = positive_lines + negative_lines
    n_positives = len(set([item[0] for item in positive_lines]))
    n_negatives = len(set([item[0] for item in negative_lines]))
    df_kltype = pd.DataFrame(lines, columns = ["phage", "depo","KLtypes"])
    return df_kltype , n_positives, n_negatives
    
def build_graph_baseline(df_info, n_positives, n_negatives) : 
    # **************************************************************
    # initialize the graph
    graph_data = HeteroData()
    # Indexation process  
    df_info = df_info[df_info["depo"].isin(df_embeddings.index)]
    indexation_nodes_A = [0]
    indexation_nodes_B1 = df_info["phage"].unique().tolist()
    indexation_nodes_B2 = df_info["depo"].unique().tolist() 
    #ID_nodes_A = {item:index for index, item in enumerate(indexation_nodes_A)}
    #ID_nodes_A_r = {index:item for index, item in enumerate(indexation_nodes_A)}
    ID_nodes_B1 = {item:index for index, item in enumerate(indexation_nodes_B1)}
    ID_nodes_B1_r = {index:item for index, item in enumerate(indexation_nodes_B1)}
    ID_nodes_B2 = {item:index for index, item in enumerate(indexation_nodes_B2)}
    ID_nodes_B2_r = {index:item for index, item in enumerate(indexation_nodes_B2)}
    # **************************************************************
    # Make the node feature file : 
    embeddings_columns = [int(i) for i in range(1, 1281)]
    node_feature_A = torch.tensor([0], dtype=torch.float)
    node_feature_B1 = torch.zeros((len(ID_nodes_B1), 0), dtype=torch.float)
    node_feature_B2 = torch.tensor([df_embeddings[df_embeddings.index == depo][embeddings_columns].values[0].tolist() for depo in df_info["depo"] if depo in df_embeddings.index] , dtype=torch.float)
    # feed the graph
    graph_data["A"].x = node_feature_A
    graph_data["B1"].x = node_feature_B1
    graph_data["B2"].x = node_feature_B2
    # **************************************************************
    # Make edge file
    # Node B2 (depolymerase) - Node B1 (prophage) :
    edge_index_B2_B1 = []
    for phage in df_info.phage.unique() :
        all_data_phage = df_info[df_info["phage"] == phage]
        all_data_phage = all_data_phage[all_data_phage["depo"].isin(df_embeddings.index)]
        for _, row in all_data_phage.iterrows() :
            edge_index_B2_B1.append([ID_nodes_B2[row["depo"]], ID_nodes_B1[row["phage"]]])
    edge_index_B2_B1 = torch.tensor(edge_index_B2_B1 , dtype=torch.long)
    # feed the graph
    graph_data['B2', 'expressed', 'B1'].edge_index = edge_index_B2_B1.t().contiguous()
    # The labels : 
    labels = [1] * n_positives + [0] * n_negatives
    graph_data["B1"].y = torch.tensor(labels)
    # Training fraction :
    train_mask = [1]* len(labels)
    graph_data["B1"].train_mask = torch.tensor(train_mask)
    return graph_data 

def train_graph(KL_type, graph_data) :
    with open(f"{path_work}/train_nn/fine_tuning/log_files_0801/{KL_type}__finetuned_node_classification.0801.log" , "w") as log_outfile :
        n_prophage = dico_prophage_count[KL_type]
        if KL_type in dico_tailored :
            model = version_dico[model_version]
        elif n_prophage <= 125 :
            model = TropiGAT_models.TropiGAT_small_module(hidden_channels = 1280, heads = 1)
            n = "small"
        else :
            model = TropiGAT_models.TropiGAT_big_module(hidden_channels = 1280, heads = 1)
            n = "big"
        model.load_state_dict(torch.load(path_models[KL_type]))
        model(graph_data)
        #
        optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001 , weight_decay= 0.000001)
        scheduler = ReduceLROnPlateau(optimizer, 'min')
        criterion = torch.nn.BCEWithLogitsLoss()
        #
        try :
            for epoch in range(10):
                train_loss = TropiGAT_models.train(model, graph_data, optimizer,criterion)
                if epoch % 1 == 0:
                    # Save the model checkpoint
                    info = f'Epoch: {epoch}\tTrain Loss: {train_loss}\n'
                    log_outfile.write(info)
                    checkpoint_path = f"{path_finetuned}/{KL_type}.{epoch}.finetuned.TropiGATv2.0801.pt"
                    torch.save(model.state_dict(), checkpoint_path)
                    print(f"Checkpoint saved: {checkpoint_path}")
        except Exception as e :
            log_outfile.write(f"***Issue here : {e}")


def finetune_kltype(kltype) :
    if kltype in KLtypes :
        #try : 
        df_kltype , n_positives, n_negatives = finetune_kltype_df(kltype)
        graph_data = build_graph_baseline(df_kltype , n_positives, n_negatives)
        #train_graph(kltype , graph_data)
        #except Exception as e :
        #    print(e , kltype)
    else :
        pass
        #with open(f"{path_work}/train_nn/fine_tuning/log_files_0801/{KL_type}__finetuned_node_classification.0801.log" , "w") as log_outfile :
            #log_outfile.write("Not in the TropiGAT system")



#if __name__ == '__main__':
#    with ThreadPool(1) as p:
#        p.map(finetune_kltype, pools_kltypes)


In [12]:
df_kltype , n_positives, n_negatives = finetune_kltype_df("KL2")
df_kltype

,phage,depo,KLtypes
0,K2PH164C1,K2PH164C1__cds_23,KL2
1,K37PH164C1,K37PH164C1__cds_47,"KL2,KL37"
2,K2PH164C2,K2PH164C2__cds_24,KL2
3,K2alfa62,K2alfa62__cds_23,KL2
4,K37PH164C1,K37PH164C1__cds_48,"KL2,KL37"
...,...,...,...
205,PP187,PP187__gp235,"KL16, KL110"
206,PP187,PP187__gp239,"KL16, KL110"
207,PP187,PP187__gp237,"KL16, KL110"
208,PP187,PP187__gp230,"KL16, KL110"


In [13]:
graph_data = build_graph_baseline(df_kltype , n_positives, n_negatives)

In [15]:
df_embeddings.index = df_embeddings.index.to_series().apply(lambda x: x.split("_Dpo")[0] if "Dpo" in x else x)
df_embeddings

,1,2,3,4,5,6,7,8,9,10,...,1271,1272,1273,1274,1275,1276,1277,1278,1279,1280
PP187__gp233,0.016811,0.073770,-0.009818,-0.036033,-0.085370,-0.016310,0.007661,-0.083562,0.072318,0.050186,...,-0.004649,-0.006715,0.121524,-0.004321,0.019053,0.074084,0.042733,-0.064831,0.040088,0.062354
LCALLHIG__00050,0.015476,0.014564,0.016806,-0.042899,-0.071834,0.021123,0.059287,-0.088285,0.081127,0.022274,...,0.016373,-0.006337,0.073145,-0.024415,0.028782,0.048048,0.012546,-0.085356,-0.006697,0.106924
NBNDMPCG__00165,0.097851,-0.005817,0.044202,0.066516,-0.062639,0.012461,-0.056078,-0.046124,0.025547,0.090553,...,0.091665,0.004563,0.011115,0.032996,0.009146,0.084371,0.129982,-0.089394,-0.080854,-0.026717
EKPIEFBL__00177,0.062666,-0.010341,-0.000893,0.034539,-0.107698,0.021799,-0.030059,-0.039033,-0.016901,0.012905,...,0.085217,-0.028300,0.048211,0.066592,0.019395,0.056614,0.101654,-0.031980,-0.056580,0.012406
NBNDMPCG__00161,0.003200,0.046110,0.004314,-0.063110,-0.094585,-0.018906,-0.011139,-0.041858,0.066822,0.052831,...,-0.010727,-0.004597,0.068099,-0.013416,0.020944,0.050755,0.065946,-0.076776,0.066887,0.054853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
K24PH164C1__cds_55,0.039957,-0.007577,-0.037035,0.054355,-0.128855,0.044301,0.099503,-0.163834,0.076119,-0.003351,...,0.016038,0.009034,0.027130,0.023492,-0.033999,-0.025833,0.030128,-0.042748,-0.122199,0.124239
K42PH8__cds_43,0.015222,-0.034378,-0.049853,0.063044,-0.158600,0.051105,0.079608,-0.183104,0.016171,-0.001478,...,0.023938,-0.037645,0.006023,0.016808,-0.034763,-0.044617,0.048723,-0.071603,-0.118990,0.130576
K10PH82C1__cds_45,0.017037,-0.028448,-0.066188,0.038587,-0.124722,0.051205,0.086921,-0.141901,0.024835,-0.017928,...,0.022757,-0.000056,0.013690,0.011025,-0.067530,-0.004214,0.050276,-0.072483,-0.137275,0.142341
K44PH129C1__cds_45,0.010439,-0.023608,-0.016569,0.054134,-0.148860,0.043717,0.065837,-0.149970,0.020374,-0.011850,...,0.042593,-0.044361,-0.003943,0.033981,-0.043663,-0.035262,0.058799,-0.032766,-0.110497,0.138307


In [ ]:
#!/bin/bash
#BATCH --job-name=2211_finetuning__
#SBATCH --qos=short
#SBATCH --ntasks=1 
#SBATCH --cpus-per-task=16 
#SBATCH --mem=50gb 
#SBATCH --time=1-00:00:00 
#SBATCH --output=2211_finetuning__%j.log 

module restore la_base
source /storage/apps/ANACONDA/anaconda3/etc/profile.d/conda.sh
conda activate torch_geometric

python /home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/train_nn/fine_tuning/script_files/finetuning.2211.py

In [ ]:
***
